In [ ]:
# !pip install poe-api
!pip install -U poe-api-wrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.5 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=

In [ ]:
from poe_api_wrapper import PoeApi

link_client = "NF0dks8o3PAvd5BKmM_V4w%3D%3D"
client = PoeApi(link_client)

# client = PoeApi("p49wzQYujpnuLVWJ8eOKDw%3D%3D")

# Using Client with proxy (default is False)
# client = PoeApi("TOKEN_HERE", proxy=True)

**Mounted drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get data**

In [4]:
import pandas as pd

In [5]:
link_data = "/content/drive/MyDrive/largeProject/code/LLM training/Splitted Visquad /df_gen_20000.csv"
num = link_data.split('/')[-1]
num = num.split('.')[0]
num = num.split('_')[-1]

In [6]:
print(num)

20000


In [7]:
df = pd.read_csv("/content/drive/MyDrive/largeProject/code/LLM training/Splitted Visquad /df_gen_20000.csv",index_col=False)

In [ ]:
def get_prompt(context, question):
    prompt = f'''Ngữ cảnh: "{context}". Câu hỏi "{question}". Trả lời câu hỏi theo từng bước dựa vào thông tin trong ngữ cảnh trên với tối đa 100 từ.'''
    return prompt.strip()

In [ ]:
df['prompt'] = df.apply(lambda x: get_prompt(x['context'], x['question']), axis=1)

In [ ]:
df['prompt'].to_list()[:10]

In [ ]:
# print(client.get_chat_history('chinchilla'))
dict_client = client.get_chat_history()

In [ ]:
list_chat_code = []

for dict_id in dict_client['acouchy']:
  list_chat_code.append(dict_id['chatCode'])

In [ ]:
list_chat_code

['2jh1k0f339bio5tb761',
 '2jh1ip08vwjlz8o4iqu',
 '2jh1g7yzb680s8j11kr',
 '2jh1gqfr7uct0vg947h',
 '2jh1qhnlo3vmk9n4hrk',
 '2jh1o5f2rkw3vr81rxm',
 '2jh1mtd80ujndjpv23c',
 '2jh1nd4l8x2dvtpdyit',
 '2jh1x8nugxfa7qymtjz']

In [ ]:
import os
import random
from tqdm import tqdm
import time

In [ ]:
list_res = []

for index, row in tqdm(df.iterrows()):
    prompt = row['prompt']
    res = ""
    while True:
        try:
            for chunk in client.send_message("acouchy", prompt, chatCode = list_chat_code[index % len(list_chat_code)], timeout=30):
                pass
            res = chunk["text"]
            if res == "You are sending and receiving too many words in a short period of time.":
                continue
            break
        except Exception as e:
            print(e)
            if "Timed out waiting for other messages to send" in str(e):
                client = PoeApi(link_client)
                dict_chat = client.get_chat_history()["acouchy"]
                list_chat_code = [each['chatCode'] for each in dict_chat]
                time.sleep(10)
            res = ""
    res = " ".join(res.split("\n")).strip()

    list_res.append((index, res))
    if index % 50 == 0 and index != 0:
        time.sleep(10)
    if index % 100 == 0 and index != 0:
        clone_ = df.loc[:index].copy()
        clone_ = clone_.drop(columns = ['prompt'])
        clone_['answer'] = list_res
        clone_.to_csv(f'/content/drive/MyDrive/largeProject/processData/mrc_{num}_{index}.csv', index=False)
    if index % 200 == 0 and index != 0:
        for idx_chat in list_chat_code:
            amount_previous_messages = len(client.get_previous_messages("acouchy", chatCode=idx_chat, get_all=True))
            if amount_previous_messages < 50:
                continue
            client.purge_conversation("acouchy", chatCode=idx_chat, count=41)
        time.sleep(20)

2it [00:16,  7.15s/it]


Retrying request 1/3 times...


3it [01:05, 26.37s/it]


Retrying request 1/3 times...


4it [01:42, 30.41s/it]

...

607it [6:29:55, 42.67s/it]


Retrying request 1/3 times...


608it [6:30:47, 45.21s/it]


Retrying request 1/3 times...


612it [6:32:00, 20.98s/it]

In [ ]:
list_res

[(0, 'Theo ngữ cảnh, KPI quý trước là 95,9%.'),
 (1,
  'Tỷ lệ hài lòng của KH dịch vụ chuyển phát chỉ bằng 0,88 lần so với KPI đề ra.'),
 (2, 'Mục tiêu đề ra là 97,2%.')]

In [ ]:
text = "chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?"

print(text, end="", flush=True)

chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?